<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/ch12_integrationIntuition_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 12 (Intuition for integration)

---

# About this code file:

### This notebook contains full code solutions to the exercises in this book chapter. There are many correct ways to solve the exercises; this notebook provides *a* solution, not *THE* solution. Please use this code as a starting point to continue exploring and experimenting with calculus concepts and visualizations.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Exercise 12.1: Cumulative sum

In [ ]:
numbers = np.arange(1,11)

# initialize
cumsum1 = np.zeros(len(numbers),dtype=int)
cumsum2 = np.zeros(len(numbers),dtype=int)

# option 1: summing the original vector from the beginning to each index
for i in range(len(numbers)):
  cumsum1[i] = np.sum( numbers[:i+1] )

# option 2: summing the previous value from the cumulative sum vector with the current value of the original vector
cumsum2[0] = numbers[0]
for i in range(1,len(numbers)):
  cumsum2[i] = cumsum2[i-1] + numbers[i]

# option 3: list comprehension without numpy
cumsum3 = [ sum(numbers[:i+1]) for i in range(len(numbers)) ]


# print the results
print('Option 1:')
print(cumsum1), print('')

print('Option 2:')
print(cumsum2), print('')

print('Option 3:')
print(cumsum3), print('')

print('Using np.cumsum:')
print(np.cumsum(numbers))

# Exercise 12.2: Functions to compute and plot the integral

In [ ]:
# create a function that computes and outputs the derivative and integral
def derivAndIntegral(x,fx):

  # difference (discrete derivative)
  dx = x[1] - x[0]
  df = np.diff(fx) / dx
  df = np.append(df,df[-1])

  # cumulative sum (discrete integral)
  idf = np.cumsum(df) * dx

  # normalize the function
  zeroIdx = np.argmin(abs(x)) # x-array index of x=0
  idf -= idf[zeroIdx] # normalize so that idf(0)=0
  idf += fx[zeroIdx]  # then add constant from original function

  # return the calculations
  return df,idf

In [ ]:
### Confirm by creating the table

# x-axis grid and function
x = np.linspace(-1,4,6)
fx = x**2

df,idf = derivAndIntegral(x,fx)


# header row
print('  x    |   fx    |   df    |   idf')
print('-----------------------------------')

# now for the results
for i in range(len(x)):
  print(f'{x[i]:>5.2f}  |  {fx[i]:>5.2f}  |  {df[i]:>5.2f}  |  {idf[i]:>5.2f}')


# Exercise 12.3: Visualize the approximations

In [ ]:
# define a function to visualize

# this corresponds to Figure 12.7
x = np.linspace(-1,4,27)
fx = x**2


# the first additional suggestion
# x = np.linspace(-1,4,73)
# fx = x**3 + 4


# second suggestion
# x = np.linspace(-np.pi,np.pi,93)
# fx = x**3/10 - np.pi*np.exp(-x**2) + np.sin(4*x)


# apply the function
df,idf = derivAndIntegral(x,fx)

In [ ]:
_,axs = plt.subplots(1,3,figsize=(12,3.5))

# visualize the function (note: panel title is hard-coded to x**2)
axs[0].plot(x,fx,'ks',markerfacecolor='w',markersize=10,linewidth=2,alpha=.5)
axs[0].set(xlabel='$x$',ylabel='$y = f(x)$',title=r'$\bf{A}$)  $f(x) = x^2$')

# visualize the derivative
axs[1].plot(x[:-1],df[:-1],'ko',markerfacecolor='w',markersize=10,linewidth=2,alpha=.5)
axs[1].set(xlabel='$x$',ylabel='$\Delta y/\Delta x$',title=r"$\bf{B}$)  Approx. derivative")

# visualize the integral of the derivative
axs[2].plot(x,idf,'k^',markerfacecolor='w',markersize=10,linewidth=2,label='Approx. integral',alpha=.5)

# and plot the original function underneath
axs[2].plot(x,fx,'k',linewidth=3,label='Orig. func.',zorder=-3)
axs[2].set(xlabel='$x$',ylabel=r'$y = f(x) $ or $\sum df/dx$',title=r"$\bf{C}$)  Function reconstruction")
axs[2].legend()

plt.tight_layout()
plt.savefig('intint_FTC1a.png')
plt.show()

 # Exercise 12.4: Geometric approximation

In [ ]:
# function for the function
def fx(u):
  return u**2 - .5

In [ ]:
# define dx
dx = .2

# plot the function
plt.figure(figsize=(5,5))


# define the resolution
xx = np.arange(-.5,1+dx,dx)

# define the function
y = fx(xx)

# plot the function
plt.plot(xx,y,'ks-',linewidth=2,markersize=10,markerfacecolor=[.4,.4,.4])



# initialize area
area = 0

# plot rectangles
for xi in xx:

  # draw the rectangle
  plt.fill_between([xi-dx/2,xi+dx/2],[fx(xi),fx(xi)],edgecolor='k',facecolor=[.9,.9,.9])

  # sum the area
  area += abs(fx(xi)*dx)

# set the labels (after the for-loop)
plt.gca().set(xlabel='x',ylabel=r'$y = x^2-.5$')
plt.title(r'Area = %.3f when $\Delta$x=%g' %(area,dx),loc='center')

# finalize plot
plt.tight_layout()
plt.savefig('intint_ex4.png')
plt.show()

# Exercise 12.5: Improving approximations by decreasing dx

In [ ]:
# analytical area calculated in sympy (more about this in the next chapter)
sx = sym.symbols('sx')
symArea = sym.integrate(sym.Abs(sx**2-.5),(sx,-.5,1.1))

print('Exact area: ',symArea)
print('numpy approx.: ',area) # from exercise 4

In [ ]:
# resolutions
dxs = np.logspace(np.log10(.5),np.log10(.001),20)
areas = np.zeros(len(dxs))

# function bounds
bounds = [-.5,1.1]

# loop over resolutions
for i,dx in enumerate(dxs):

  # x-axis grid
  xx = np.arange(bounds[0],bounds[1]+dx,dx)

  # compute area using rectangle area formula
  area_tmp = 0
  for xi in xx: area_tmp += abs(fx(xi)*dx)

  # store final result
  areas[i] = area_tmp

In [ ]:
_,ax = plt.subplots(1,figsize=(8,4))

# plot the results
ax.plot(dxs,areas,'ks-',linewidth=2,markerfacecolor='w',markersize=10,label='Approximations')
ax.axvline(.2, linestyle=':',color=[.7,.7,.7],zorder=-1,label=r'$\Delta x$ in Exercise 3')
ax.axhline(symArea,linestyle='--',color='k',label='Exact value')

ax.invert_xaxis()
ax.set_xscale('log')
ax.set(xlabel=r'$\Delta x$',ylabel='Area (definite integral)')
ax.legend()

plt.tight_layout()
plt.savefig('intint_ex5.png')
plt.show()